# Convert Masks to YOLO Annotations
### Dataset Format for YOLO
YOLO requires annotations in .txt files (one per image) with bounding boxes or segmentation masks. For object detection (bounding boxes), use this format:
```bash
0 0.5 0.5 0.2 0.3  # class_id, x_center, y_center, width, height
1 0.7 0.3 0.1 0.1
```
If your masks are binary images, use OpenCV to extract bounding boxes:

In [5]:
import cv2
import os
import numpy as np
from pathlib import Path

def mask_to_yolo(mask_path, output_txt_path, class_id, mode='segmentation'):
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Warning: Could not read mask {mask_path}")
        return
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    with open(output_txt_path, 'w') as f:
        for contour in contours:
            if mode == 'segmentation':
                contour = contour.squeeze()
                if contour.ndim == 1:
                    continue  # Skip invalid contours
                contour_norm = contour / np.array([mask.shape[1], mask.shape[0]])
                f.write(f"{class_id} " + " ".join([f"{x:.5f} {y:.5f}" for x,y in contour_norm]) + "\n")
            elif mode == 'detection':
                x, y, w, h = cv2.boundingRect(contour)
                x_center = (x + w/2) / mask.shape[1]
                y_center = (y + h/2) / mask.shape[0]
                width = w / mask.shape[1]
                height = h / mask.shape[0]
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {width:.5f} {height:.5f}\n")

# Process all splits (match your folder names: train, valid, test)
for split in ['train', 'valid', 'test']:  # Fixed 'val' -> 'valid'
    split_dir = Path(f'/home/user/Documents/LRS/mask/{split}')
    image_dir = split_dir / 'images'
    label_dir = split_dir / 'labels'
    
    # Create both images and labels directories
    image_dir.mkdir(parents=True, exist_ok=True)  # Fix: Create images directory
    label_dir.mkdir(parents=True, exist_ok=True)
    
    # Process all masks in the split directory
    for mask_path in split_dir.glob('*_mask.png'):
        # Get corresponding image (e.g., xyz_mask.png → xyz.jpg)
        image_path = mask_path.with_name(mask_path.name.replace('_mask.png', '.jpg'))
        
        # Move image to images subdirectory
        try:
            image_path.rename(image_dir / image_path.name)
        except FileNotFoundError:
            print(f"Error: Image {image_path} not found. Skipping...")
            continue
        
        # Generate YOLO annotation file
        output_txt = label_dir / f"{mask_path.stem.replace('_mask', '')}.txt"
        mask_to_yolo(mask_path, output_txt, class_id=0, mode='segmentation')  # Adjust mode as needed

# Segmentation (YOLOv8)

Install YOLOv8:

In [6]:
!pip3 install ultralytics

  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp312-cp312-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.

## Training Code

In [8]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')  # Segmentation model
model.train(data='/home/user/Documents/LRS/data.yaml', epochs=50, imgsz=640, batch=16)

Ultralytics 8.3.69 🚀 Python-3.12.8 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7940MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/home/user/Documents/LRS/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

100%|██████████| 5.35M/5.35M [01:12<00:00, 77.9kB/s]


AMP: checks passed ✅


train: Scanning /home/user/Documents/LRS/mask/train/labels... 105 images, 0 backgrounds, 0 corrupt: 100%|██████████| 105/105 [00:00<00:00, 2171.97it/s]

train: New cache created: /home/user/Documents/LRS/mask/train/labels.cache


...

                   all         11         54      0.566      0.611      0.562      0.307      0.532      0.611      0.543      0.271



50 epochs completed in 0.019 hours.
Optimizer stripped from /home/user/Documents/RSI/runs/segment/train2/weights/last.pt, 6.8MB
Optimizer stripped from /home/user/Documents/RSI/runs/segment/train2/weights/best.pt, 6.8MB

Validating /home/user/Documents/RSI/runs/segment/train2/weights/best.pt...
Ultralytics 8.3.69 🚀 Python-3.12.8 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7940MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,454 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 14.75it/s]


                   all         11         54      0.664      0.463      0.571      0.343      0.711      0.426      0.545       0.28
                  road         11         54      0.664      0.463      0.571      0.343      0.711      0.426      0.545       0.28
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/user/Documents/RSI/runs/segment/train2


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7826d43b1580>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [ ]:
from ultralytics import YOLO
import cv2

# Load the trained model
model = YOLO('/home/user/Documents/RSI/runs/segment/train2/weights/best.pt')

# Predict on a single image
image_path = "/home/user/Documents/LRS/mask/test/images/tile_12_jpg.rf.b0d051c082e97538bdd45a416fa6d93b.jpg"
results = model.predict(image_path, imgsz=640, conf=0.5)

# Visualize results
for result in results:
    plotted = result.plot()  # Draw masks/boxes on the image
    cv2.imshow("Prediction", plotted)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


image 1/1 /home/user/Documents/LRS/mask/test/images/tile_12_jpg.rf.b0d051c082e97538bdd45a416fa6d93b.jpg: 640x640 6 roads, 3.6ms
Speed: 1.2ms preprocess, 3.6ms inference, 268.1ms postprocess per image at shape (1, 3, 640, 640)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/user/Documents/LRS/.venv/lib/python3.12/site-packages/cv2/qt/plugins"
